# Basic setting

In [2]:
import pandas as pd
from glob import glob
import fasttext
from gensim.models import Word2Vec, FastText
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.metrics.pairwise import cosine_similarity

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Load data

In [3]:
dict_data = {}
for i in glob('data/*.csv'):
    try:
        dict_data[i.split('/')[1].split('.csv')[0]] = pd.read_csv(i)
    except:
        dict_data[i.split('/')[1].split('.csv')[0]] = pd.read_csv(i, encoding='cp949')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (2,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df = dict_data['suggest ver4.0 (memebers_added)']
df

,BILL_ID,BILL_NO,BILL_NAME,COMMITTEE,PROPOSE_DT,PROC_RESULT,AGE,DETAIL_LINK,PROPOSER,MEMBER_LIST,RST_PROPOSER,PUBL_PROPOSER,bill_content,senators_list
0,PRC_A2T0Z0E9U1I6P1G3P3X9I3B7H7D9C1,2104402,독점규제 및 공정거래에 관한 법률 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,박홍근의원등12인,http://likms.assembly.go.kr/bill/coactorListPo...,박홍근,"김원이,남인순,서영석,송옥주,용혜인,윤미향,이용빈,장경태,정태호,주철현,최종윤",「독점규제 및 공정거래에 관한 법률」(이하 “공정거래법”이라 함)은 시장경제의 기본...,최종윤(더불어민주당/崔鍾允) 주철현(더불어민주당/朱哲鉉) 정태호(더불어민주당/鄭泰浩...
1,PRC_I2M0O1G0U0P5E1U8T4B6Z0R9M8G5X0,2104401,지방교육재정교부금법 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,서영교의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,서영교,"김병기,서삼석,신정훈,오영환,윤재갑,이개호,이성만,전혜숙,최형두",제안이유 및 주요내용 현행법 제3조제2항은 지방교육재정교부금 재원을 내국세 총액에...,최형두(국민의힘/崔炯斗) 전혜숙(더불어민주당/全惠淑) 이성만(더불어민주당/李成萬) ...
2,PRC_C2Q0L1M0V0B5G1O8H1N6Z5V0W5W7A7,2104400,농업기계화 촉진법 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,윤재갑의원 등 11인,http://likms.assembly.go.kr/bill/coactorListPo...,윤재갑,"강병원,김수흥,서영교,위성곤,이개호,이재정,이정문,전용기,조경태,홍문표",제안이유 및 주요내용 현행법은 농업기계의 제조·수입업자는 다른 용도의 기계와 구분...,홍문표(국민의힘/洪文杓) 조경태(국민의힘/趙慶泰) 전용기(더불어민주당/田溶冀) 이정...
3,PRC_F2A0Q1V0F0S5O1N8W1P6L2Z9T9D6K3,2104399,지방교부세법 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,서영교의원 등 10인,http://likms.assembly.go.kr/bill/coactorListPo...,서영교,"김병기,신정훈,오영환,윤재갑,윤준병,이개호,이성만,전혜숙,최형두",제안이유 및 주요내용 현행 소방공무원의 국가직 전환 이후 소방안전교부세율은 담배에...,최형두(국민의힘/崔炯斗) 전혜숙(더불어민주당/全惠淑) 이성만(더불어민주당/李成萬) ...
4,PRC_C2N0L1B0M0O5S1P8T0O6I1O8S4J8O7,2104398,농수산물의 원산지 표시에 관한 법률 일부개정법률안,NaN,2020-10-05,NaN,21,http://likms.assembly.go.kr/bill/billDetail.do...,윤재갑의원 등 11인,http://likms.assembly.go.kr/bill/coactorListPo...,윤재갑,"강병원,김수흥,서영교,위성곤,이개호,이재정,이정문,전용기,조경태,홍문표",제안이유 및 주요내용 현행법에 따르면 식품접객업 및 집단급식소 중 대통령령령으로 ...,홍문표(국민의힘/洪文杓) 조경태(국민의힘/趙慶泰) 전용기(더불어민주당/田溶冀) 이정...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62653,000021,10025,반민족행위특별재판부부속기관조직법안,NaN,1948-11-23,원안가결,1,http://likms.assembly.go.kr/bill/billDetail.do...,김상덕의원외9인,http://likms.assembly.go.kr/bill/coactorListPo...,NaN,NaN,■ 제안이유反民族行爲特別裁判部 및 特別檢察部의 附屬機關의 組織에 관한 規定을 두어 ...,내용없음
62654,000022,10024,반민족행위처벌법중개정법률안,NaN,1948-11-23,원안가결,1,http://likms.assembly.go.kr/bill/billDetail.do...,김상덕의원외9인,http://likms.assembly.go.kr/bill/coactorListPo...,NaN,NaN,■ 제안이유特別調査委員의 調査業務를 圖滑히 하기 위하여 調査上 必要할 境遇 司法警察...,내용없음
62655,000018,10021,농지개혁법안,NaN,1948-11-13,대안반영폐기,1,http://likms.assembly.go.kr/bill/billDetail.do...,이훈구의원외31인,http://likms.assembly.go.kr/bill/coactorListPo...,NaN,NaN,■ 제안이유 및 주요골자資料없음.資料없음.■ 기타사항第2回國會(定期會) 産業委員會(...,내용없음
62656,000014,10015,한글전용에관한법률안,NaN,1948-09-29,원안가결,1,http://likms.assembly.go.kr/bill/billDetail.do...,권태희의원외138인,http://likms.assembly.go.kr/bill/coactorListPo...,NaN,NaN,■ 제안이유한 나라의 國語에는 그 나라의 國民精神이 흐르고 있고 民族正氣가 內包되어...,내용없음


# Install FastText

!wget https://github.com/facebookresearch/fastText/archive/0.2.0.zip
!unzip 0.2.0.zip
%cd fastText-0.2.0
!make

# Build corpus

In [5]:
mecab = Mecab()
def tokenized_mecab(data):
    result = mecab.morphs(data)
    result = ' '.join(result)
    return result

In [6]:
def out_stopwords(data, list_stopwords):
    data = data.split(" ")
    list_stopwords = list_stopwords
    data = [token for token in data if token not in list_stopwords]
    return data

In [7]:
%%time
df['tokenized'] = df['bill_content'].progress_apply(tokenized_mecab)


CPU times: user 55 s, sys: 745 ms, total: 55.7 s
Wall time: 55.6 s


In [8]:
url= "/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/코딩 관련/개인프로젝트_국회의원 추천 (모델)/1차_stopwords 추가/stopwords ver1.0.txt"

stopwords_file = list(open(url,"r"))
list_stopword = []
for i in stopwords_file:
    list_stopword.append(i[:-1])

list_stopword.append('제안이유')
list_stopword.append('주요')
list_stopword.append('내용')

In [9]:
list_stopword

['정',
 '정애',
 '정정보',
 '정주',
 '정하',
 '제원',
 '져',
 '져야',
 '졌',
 '졌으며',
 '존',
 '좁',
 '좋',
 '주운',
 '준',
 '준다는',
 '줄',
 '줄어드',
 '줄여',
 '줄어들',
 '줌',
 '즐길',
 '즐기',
 '지',
 '지나',
 '지나치',
 '지나친',
 '지난',
 '지니',
 '지득',
 '지로',
 '지만',
 '지발',
 '지소',
 '지켜',
 '지켜야',
 '지키',
 '지킬',
 '직',
 '진',
 '진행할',
 '짐',
 '짓',
 '징',
 '짧',
 '째',
 '쪽',
 '찔려',
 '찰',
 '참',
 '찾',
 '찾악',
 '찾아보',
 '챗',
 '처',
 '처럼',
 '청',
 '초',
 '촌',
 '첫째',
 '촌정',
 '추',
 '축',
 '출',
 '충실하',
 '충실히',
 '취',
 '측',
 '취한',
 '취할',
 '취해',
 '취해야',
 '치러',
 '치르',
 '치를',
 '카',
 '커서',
 '커져',
 '커지',
 '커진',
 '커질',
 '커짐',
 'AI ',
 'ASF ',
 'Act  ',
 'C ',
 'D ',
 'Feed ',
 'G ',
 'ILO ',
 'K',
 'KS ',
 'M  ',
 'MRG ',
 'Me  ',
 'P ',
 'PC ',
 'PM ',
 'PP ',
 'R  ',
 'SOC ',
 'TV ',
 'Too ',
 'U ',
 'UN ',
 'VOD  ',
 '` ',
 'a ',
 'and ',
 'b ',
 'ba ',
 'back ',
 'c ',
 'cc ',
 'ce ',
 'd ',
 'de ',
 'df ',
 'e ',
 'ea ',
 'f ',
 'fb ',
 'fc ',
 'ha ',
 'kW ',
 'kWh ',
 'kg ',
 'km ',
 'm ',
 'of ',
 'p ',
 '켜',
 '케',
 '크',
 '큰',
 '키워',
 '키우',
 '큼',
 '킹',
 '타

In [10]:
%%time
df['cleared'] = df['tokenized'].progress_apply(lambda x: out_stopwords(x, list_stopwords = list_stopword))


CPU times: user 5min 55s, sys: 3.6 s, total: 5min 59s
Wall time: 5min 59s


In [11]:
df['cleared'] = [' '.join(map(str, l)) for l in df['cleared']]

In [12]:
df.to_csv('data/suggest ver5.0 (bills content cleared).csv', index=False)

# Get corpus

In [13]:
tokenized = ''
for content in df['cleared'].values:
    tokenized = tokenized + "{} \n".format(content)
    
f = open("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/코딩 관련/개인프로젝트_국회입법시각화/data/corpus ver3.0 (전체, stopword 제거).txt", 'w')
f.write(tokenized)
f.close()

In [14]:
direc ='/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/코딩 관련/개인프로젝트_국회입법시각화/data/corpus ver3.0 (전체, stopword 제거).txt'

corpus = [sent.strip().split(" ") for sent in open(direc).readlines()]

# Train Fasttext (iteration 100)

In [15]:
%%time
model_gs = FastText()

model_gs.build_vocab(sentences = corpus)

CPU times: user 13.7 s, sys: 3.84 s, total: 17.5 s
Wall time: 18.3 s


In [16]:
%%time
model_gs.train(sentences = corpus, total_examples = len(corpus), epochs=100)

CPU times: user 45min 45s, sys: 15.8 s, total: 46min 1s
Wall time: 15min 33s


### Save model

In [18]:
model_gs.save("/Users/junghyunwoo/혀누에-의한-혀누를-위한-혀누의/공부공부공부공부/대학교/코딩 관련/개인프로젝트_국회입법시각화/models/FasttextModel ver4.0 (전체, stopwords 제거, iteration 100).bin")

### Load model and test

In [19]:
model_test = FastText.load("models/FasttextModel ver4.0 (전체, stopwords 제거, iteration 100).bin")

In [20]:
model_test.wv.most_similar('성폭행')

[('성추행', 0.7576199769973755),
 ('살해', 0.7201322317123413),
 ('강남역', 0.6570789813995361),
 ('성범죄', 0.6550352573394775),
 ('폭행', 0.6400604844093323),
 ('유괴', 0.6330398321151733),
 ('납치', 0.6195483207702637),
 ('성폭력', 0.6071250438690186),
 ('살인', 0.6055281758308411),
 ('ㆍ성폭력', 0.5852952003479004)]

In [21]:
model_test.wv.most_similar('원자력')

[('원자력법', 0.6811627745628357),
 ('원자력손해', 0.6601952314376831),
 ('원전', 0.5897300243377686),
 ('원자로', 0.5580499768257141),
 ('원자', 0.5535094141960144),
 ('방사선', 0.5026422142982483),
 ('방사성', 0.49495548009872437),
 ('유전자변형생물체', 0.4613633453845978),
 ('금관리', 0.44524967670440674),
 ('방사선량', 0.4441271424293518)]

In [22]:
model_test.wv.most_similar('원자력')

[('원자력법', 0.6811627745628357),
 ('원자력손해', 0.6601952314376831),
 ('원전', 0.5897300243377686),
 ('원자로', 0.5580499768257141),
 ('원자', 0.5535094141960144),
 ('방사선', 0.5026422142982483),
 ('방사성', 0.49495548009872437),
 ('유전자변형생물체', 0.4613633453845978),
 ('금관리', 0.44524967670440674),
 ('방사선량', 0.4441271424293518)]

In [23]:
model_test.wv.most_similar("장애")

[('장애인', 0.8145902156829834),
 ('노인', 0.6011108160018921),
 ('장애인복지시설', 0.5970298647880554),
 ('ㆍ장애인', 0.5899784564971924),
 ('장애인복지법', 0.5881521105766296),
 ('고령자', 0.5867133140563965),
 ('장애자', 0.585363507270813),
 ('장애인자립생활지원센터', 0.5691983699798584),
 ('애인', 0.5530550479888916),
 ('중증', 0.5477349162101746)]

In [26]:
model_test.wv.most_similar("코로나")

[('감염증', 0.7181985378265381),
 ('바이러스', 0.6645189523696899),
 ('COVID', 0.5910235047340393),
 ('팬데', 0.5757613778114319),
 ('메르스', 0.5747332572937012),
 ('MERS', 0.574539065361023),
 ('경제위기', 0.5739991068840027),
 ('구제역', 0.5704492330551147),
 ('유행', 0.5703896284103394),
 ('여파', 0.5625848174095154)]